In [1]:
from pyomo.environ import *
import pyomo.opt

In [2]:
m = ConcreteModel()
m.bin = Var(domain=Binary)
m.b = Var(domain=PositiveReals)
m.c = Var(domain=PositiveReals)
m.d = Var(domain=Integers)
m.q = Var(domain=Reals)
m.constraint4 = Constraint(rule=lambda model: model.q - log(model.c) == 0)
m.constraint1 = Constraint(rule=lambda model: model.c + model.b <= 11)
m.constraint3 = Constraint(rule=lambda model: 3 <= model.c <= 5)
m.constrain2 = Constraint(rule=lambda model: 1 <= model.d <= 5)
m.OBJ = Objective(rule=lambda model: (model.bin * model.q) , sense=maximize)
solver = pyomo.opt.SolverFactory('bonmin', executable='/Users/ashton/school/cmsc828m/project/code/Bonmin-1.8.6.backup/bin/bonmin')
solver.solve(m, tee=True, keepfiles=True)

Solver log file: '/var/folders/bp/mm3fnf9j6fg_kwpplwd4dzyw0000gn/T/tmpbi1kn7_bonmin.log'
Solver solution file: '/var/folders/bp/mm3fnf9j6fg_kwpplwd4dzyw0000gn/T/tmpCAUoXG.pyomo.sol'
Solver problem files: ('/var/folders/bp/mm3fnf9j6fg_kwpplwd4dzyw0000gn/T/tmpCAUoXG.pyomo.nl',)
Bonmin 1.8.6 using Cbc 2.9.9 and Ipopt 3.12.8
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -1.6094379       10 0.004578
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I        

{'Problem': [{'Number of objectives': 1, 'Lower bound': -inf, 'Number of variables': 5, 'Upper bound': inf, 'Sense': 'unknown', 'Number of constraints': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Time': 0.15975499153137207, 'Message': 'bonmin\\x3a Optimal', 'Error rc': 0, 'Id': 3}]}

In [3]:
m.bin.value, m.c.value, m.q.value

(1.0, 5.00000004995, 1.6094379224241002)

# Real formulation

In [2]:
import pickle

In [3]:
rewards = pickle.load(open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180411-rewards.pickle', 'rb'))
for i in range(len(rewards)):
    for j in range(len(rewards[i])):
        rewards[i][j].append({'bs': 0, 'cve_id': 'NULL', 'es': 0, 'is': 0})
        


In [4]:
#OPTION 1
modified_rewards = []
for i in range(len(rewards)):
    for j in range(len(rewards[i])):
        for k in range(len(rewards[i][j])):
            modified_rewards.append((i,j,k))
NUM_ATTACKERS = 3
NUM_SERVICES = 4
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
x = [0.1, 0.1, 0.1, 0.1]

In [5]:
#OPTION 2
modified_rewards = []
for i in range(2):
    for j in range(len(rewards[i])):
        for k in range(len(rewards[i][j])):
            modified_rewards.append((i,j,k))
NUM_ATTACKERS = 3
NUM_SERVICES = 2
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
x = [0.1, 0.1, 0, 0]

In [8]:
class Experiment:
    
    def __init__(self, attacker_prob, service_prob, rewards):
        self.attacker_prob = attacker_prob
        # changing x to self.c
        self.c = service_prob
        self.rewards = rewards
        
        self.modified_rewards = []
        for i in range(len(rewards)):
            for j in range(len(rewards[i])):
                for k in range(len(rewards[i][j])):
                    self.modified_rewards.append((i,j,k))
                    
        self.m = ConcreteModel()
    
    def init_vars(self):
        self.m.attackers = Set(initialize=list(range(len(self.attacker_prob))))
        self.m.service_set = Set(initialize=list(range(len(self.c))))
        self.m.attacks = Set(initialize=self.modified_rewards, dimen=3)
        
        self.m.h = Var(self.m.service_set, domain=PositiveReals)
        self.m.n = Var(self.m.attacks, domain=Binary)
        self.m.v = Var(self.m.attackers, domain=Reals)
        self.m.q = Var(self.m.service_set, domain=PositiveReals)
    
    def set_obj(self):
        
        def obj_rule(model):
    #     return sum(attacker_prob[theta] * m.n[(s, theta, a)] * exp(m.q[s]) * (m.x_prime[s] * rewards[s][theta][a]['is']) \
    #                for s, theta, a in m.attacks)
            return sum(self.attacker_prob[theta] * model.n[(s, theta, a)] * exp(model.q[s]) * \
                   (self.c[s] * -1 * self.rewards[s][theta][a]['is'] + \
                    LOSS_AVERSION_FACTOR * model.h[s] * self.rewards[s][theta][a]['is']) \
                   for s, theta, a in model.attacks)
    
        self.m.OBJ = Objective(rule=obj_rule, sense=maximize)
    
    def set_constraints(self):
        def h_positive(model, s):
            return model.h[s] >= 0
        self.m.h_pos_constr = Constraint(self.m.service_set, rule=h_positive)

        def x_sum(model):
            return sum(model.h[s] + self.c[s] for s in model.service_set) == 1
        self.m.x_sum = Constraint(rule=x_sum)

        def best_attacker_u1(model, s, theta, a):
            #s, theta, a = attack
            return model.v[theta] - exp(model.q[s]) * (self.rewards[s][theta][a]['bs'] * self.c[s] + \
                                          -1 * ATTACKER_LOSS_FACTOR * self.rewards[s][theta][a]['es']  * model.h[s]) \
                                        >= 0.001
        #     return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s]) \
        #             >= -0.001
        self.m.best_attacker_constr1 = Constraint(self.m.attacks, rule=best_attacker_u1)
        
        def best_attacker_u2(model, s, theta, a):
        #s, theta, a = attack
            return model.v[theta] - exp(model.q[s]) * (self.rewards[s][theta][a]['bs'] * self.c[s] \
                                          + -1 * ATTACKER_LOSS_FACTOR * self.rewards[s][theta][a]['es']  * model.h[s]) \
                    <= (1 - model.n[(s, theta, a)]) * M + 0.001
        #     return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s]) \
        #             <= (1 - m.n[(s, theta, a)]) * M + .001
        self.m.best_attacker_constr2 = Constraint(self.m.attacks, rule=best_attacker_u2)           

        def only_one_action(model, attacker):
            return sum(model.n[(s, theta, a)] for s, theta, a in model.attacks if theta == attacker) == 1

        self.m.only_one_action_constr = Constraint(self.m.attackers, rule=only_one_action)

        def q_rule(model, s):
            return model.q[s] == -1 * log(self.c[s] + model.h[s])
        self.m.q_constr = Constraint(self.m.service_set, rule=q_rule)
    
    def solve(self, tee_p=False):
        solver = pyomo.opt.SolverFactory('bonmin', executable='/Users/ashton/school/cmsc828m/project/code/Bonmin-1.8.6.backup/bin/bonmin')
        solver.solve(self.m, tee=tee_p)
    
    def print_results(self):
        print("Attacks")
        print("(service, attacker, attack), was_selected, CVE Info")
        for n_i in self.m.n:
            if self.m.n[n_i].value > 0:
                s, theta, a = n_i
                print (n_i, self.m.n[n_i].value, self.rewards[s][theta][a])
                
        print()
        print("Obj: ", self.m.OBJ())
        
        print()
        print("Service Honeypot Distribution:")
        for s in self.m.service_set:
            print((s, self.m.h[s].value))
        

In [9]:
rewards = pickle.load(open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180411-rewards.pickle', 'rb'))
e = Experiment(attacker_prob=[0.5, 0.4, 0.1], service_prob=[0.1, 0.1, 0.1, 0.1], rewards=rewards)
e.init_vars()
e.set_obj()
e.set_constraints()
e.solve()


In [11]:
e.print_results()

Attacks
(service, attacker, attack), was_selected, CVE Info
((2, 0, 1), 1.0, {'is': 10.0, 'es': 3.9, 'cve_id': u'CVE-2017-12172', 'bs': 7.2})
((2, 1, 3), 1.0, {'is': 10.0, 'es': 3.9, 'cve_id': u'CVE-2017-12172', 'bs': 7.2})
((1, 2, 3), 1.0, {'is': 10.0, 'es': 10.0, 'cve_id': u'CVE-2015-8880', 'bs': 10.0})
()
('Obj: ', -2.524509271048291)
()
Service Honeypot Distribution:
(0, 0.1444661521071574)
(1, 0.06309306050548959)
(2, 0.24736260594271559)
(3, 0.14507818144463744)


In [216]:
m = ConcreteModel()

# attacker distribution
m.attackers = Set(initialize=list(range(NUM_ATTACKERS)))
attacker_prob = [0.5, 0.4, 0.1]

#required services (proportion of network for now)
m.service_set = Set(initialize=list(range(NUM_SERVICES)))
m.attacks = Set(initialize=modified_rewards, dimen=3)
#m.x = Set(initialize=[0.1, 0.08, 0.06, 0.04])
#x = {'python' : 0.1, 'mongodb' : 0.08, 'mysql' : 0.06, 'php': 0.04}


m.x_prime = Var(m.service_set, domain=PositiveReals)
m.n = Var(m.attacks, domain=Binary)
m.v = Var(m.attackers, domain=Reals)
m.q = Var(m.service_set, domain=PositiveReals)

In [217]:
# Create objective
#def obj_rule(m):
    #return sum(m.Y[e] * self.arc_data.ix[e,'Cost'] for e in self.arc_set)
#self.m.OBJ = pe.Objective(rule=obj_rule, sense=pe.minimize)
def obj_rule(model):
#     return sum(attacker_prob[theta] * m.n[(s, theta, a)] * exp(m.q[s]) * (m.x_prime[s] * rewards[s][theta][a]['is']) \
#                for s, theta, a in m.attacks)
    return sum(attacker_prob[theta] * m.n[(s, theta, a)] * exp(m.q[s]) * \
               (x[s] * -1 * rewards[s][theta][a]['is'] + \
                LOSS_AVERSION_FACTOR * m.x_prime[s] * rewards[s][theta][a]['is']) \
               for s, theta, a in m.attacks)
m.OBJ = Objective(rule=obj_rule, sense=maximize)

In [218]:
def x_prime_positive(model, s):
    return model.x_prime[s] >= 0
m.x_prime_constr1 = Constraint(m.service_set, rule=x_prime_positive)

def x_sum(model):
    return sum(model.x_prime[s] + x[s] for s in model.service_set) == 1
m.x_sum = Constraint(rule=x_sum)

def best_attacker_u1(model, s, theta, a):
    #s, theta, a = attack
    return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s] + \
                                  -1 * ATTACKER_LOSS_FACTOR * rewards[s][theta][a]['es']  * m.x_prime[s]) \
                                >= 0.001
#     return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s]) \
#             >= -0.001

def best_attacker_u2(model, s, theta, a):
#s, theta, a = attack
    return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s] \
                                  + -1 * ATTACKER_LOSS_FACTOR * rewards[s][theta][a]['es']  * m.x_prime[s]) \
            <= (1 - m.n[(s, theta, a)]) * M + 0.001
#     return m.v[theta] - exp(m.q[s]) * (rewards[s][theta][a]['bs'] * x[s]) \
#             <= (1 - m.n[(s, theta, a)]) * M + .001
    
m.best_attacker_constr1 = Constraint(m.attacks, rule=best_attacker_u1)
m.best_attacker_constr2 = Constraint(m.attacks, rule=best_attacker_u2)           

def only_one_action(model, attacker):
    return sum(model.n[(s, theta, a)] for s, theta, a in model.attacks if theta == attacker) == 1
        
m.only_one_action_constr = Constraint(m.attackers, rule=only_one_action)

def q_rule(model, s):
    return model.q[s] == -1 * log(x[s] + model.x_prime[s])

m.q_constr = Constraint(m.service_set, rule=q_rule)

In [219]:
solver = pyomo.opt.SolverFactory('bonmin', executable='/Users/ashton/school/cmsc828m/project/code/Bonmin-1.8.6.backup/bin/bonmin')
solver.solve(m, tee=True)

Bonmin 1.8.6 using Cbc 2.9.9 and Ipopt 3.12.8
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -6.6761298e-07       22 0.023864
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I           201      INFEAS 1       11 0.011665
NLP0014I             2         OPT -6.6761298e-07       23 0.026306
NLP0014I             3      INFEAS 0.86002157       30 0.040779
NLP0014I             4         OPT -6.6761298e-07       23 0.031815
NLP0014I             5      INFEAS

{'Problem': [{'Number of objectives': 1, 'Lower bound': -inf, 'Number of variables': 166, 'Upper bound': inf, 'Sense': 'unknown', 'Number of constraints': 0}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Time': 84.52261900901794, 'Message': 'bonmin\\x3a Optimal', 'Error rc': 0, 'Id': 3}]}

In [220]:
for n_i in m.n:
    if m.n[n_i].value > 0:
        s, theta, a = n_i
        print (n_i, m.n[n_i].value, rewards[s][theta][a])

((1, 2, 14), 1.0, {'is': 10.0, 'es': 10.0, 'cve_id': u'CVE-2016-2554', 'bs': 10.0})
((2, 0, 1), 1.0, {'is': 10.0, 'es': 3.9, 'cve_id': u'CVE-2017-12172', 'bs': 7.2})
((2, 1, 3), 1.0, {'is': 10.0, 'es': 3.9, 'cve_id': u'CVE-2017-12172', 'bs': 7.2})


In [212]:
m.OBJ()

-2.5245092710410058

In [213]:
for s in m.service_set:
    print((s, m.x_prime[s].value))

(0, 0.14446615210753033)
(1, 0.06309306050211068)
(2, 0.2473626059453066)
(3, 0.14507818144505236)


In [214]:
m.v[0].value, m.v[1].value, m.v[2].value

(1.2410906037399718, 1.2415857821228018, 4.972627386729124)

In [204]:
m.q[0].value

1.7056947275683925